# Question 9

In [14]:
import tensorflow as tf
import numpy as np
from itertools import product

In [15]:
learning_rate=0.3
no_actions=2
no_states=5
no_hidden=20
discountFactor = 0.5
e = 0.1
num_episodes = 5000
list=[]

In [16]:
input_data=tf.placeholder(tf.float32, shape=(1, no_states))
weight_1 = tf.Variable(tf.random_uniform(shape=[no_states,no_hidden]))
weight_2=tf.Variable(tf.random_uniform(shape=[no_hidden,no_actions]))
bias_1=tf.Variable(tf.zeros([no_hidden]))
bias_2=tf.Variable(tf.zeros([no_actions]))
output_action= tf.placeholder(tf.float32, shape=(1, 2))

hidden_layer=tf.add(tf.matmul(input_data,weight_1),bias_1)
hidden_layer=tf.nn.sigmoid(hidden_layer)
output=tf.add(tf.matmul(hidden_layer,weight_2),bias_2)

predictedValues = tf.argmax(output,1)
loss = 0.5*(output_action - output)**2
trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss=loss)

In [17]:
def tau(s,a):
    if s == 0 or s==4:
        return 0
    if a==1:
        return s+1
    else:
        return s-1

def rho(s,a):
    if s==1 and a==0:
        return 1
    if s==3 and a==1:
        return 2
    return 0

def calc_policy(Q):
    policy=np.zeros(5)
    for s in range(0,5):
        action_idx=np.argmax(Q[s,:])
        policy[s]=2*action_idx-1
        policy[0]=policy[4]=0
    return policy.astype(int)

In [18]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(num_episodes):
        for s,a in product(range(no_states),range(no_actions)):
            # j=j+1
            _, allQ = sess.run([predictedValues, output], feed_dict={input_data: np.identity(no_states)[s:s + 1]})
            if np.random.rand() < e:
                a = 1-a

            sNew = tau(s, a)
            r = rho(s, a)
            Q1 = sess.run(output, feed_dict={input_data: np.identity(no_states)[sNew:sNew + 1]})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0, a] = r + discountFactor * maxQ1
            if i>500:
                pass
            sess.run([trainer], feed_dict={input_data: np.identity(5)[s:s + 1], output_action: targetQ})
            s = sNew
    for s in range(no_states):
         list.append(sess.run(output,feed_dict={input_data:np.identity(5)[s:s+1]})[0])
    Q=np.array(list)
    print "Q table is given as ",Q

    print "Best policy is given as ",calc_policy(Q)

Q table is given as  [[0.  0. ]
 [1.  0.5]
 [0.5 1. ]
 [0.5 2. ]
 [0.  0. ]]
Best policy is given as  [ 0 -1  1  1  0]


## REFERENCES
I have taken reference from the given site for implementing this code. 
https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0